In [1]:
import pyodbc 
from pymongo import MongoClient
import subprocess
import pandas as pd
from datetime import datetime
import time
import os
from PIL import Image
import io
from bson.code import Code
import json
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', 1000)

## MONGO

In [51]:
USERNAME_MONGO = 'mongo'
PASSWORD_MONGO = 'Mi_Super_Mega_Clave_Mongo'
HOSTNAME_MONGO = 'mongo'
PORT_MONGO = '27017'
DATABASE_MONGO = 'test'
TABLE_MONGO_DATA = 'Customer_Data'
TABLE_MONGO_HEAD = 'Customer_Head'

In [69]:
## ME CONECTO A LA BASE DE DATOS DE MONGODB
MONGO_URI = 'mongodb://'+USERNAME_MONGO+':'+PASSWORD_MONGO+'@'+HOSTNAME_MONGO+':'+PORT_MONGO
## ME CONECTO AL CLIENTE DE MONGODB 
client = MongoClient(MONGO_URI)
## ME CONECTO A LA TABLA ALERTS EN LA BASE DE DATOS TEST
db_d = client[DATABASE_MONGO][TABLE_MONGO_DATA]
db_h = client[DATABASE_MONGO][TABLE_MONGO_HEAD]
db_d.drop()
db_h.drop()

In [ ]:
#mongo --port 27017  --authenticationDatabase "admin" -u "mongo"
#use test
#show collections
# db.Person.count()

## GUARDAR EN MONGO

In [190]:
records = json.loads(df_sql.to_json()).values()
db_d.insert(records)
headers = json.loads(df_sql.head(1).to_json())
db_h.insert(headers)
l_headers = list([i.keys() for i in db_h.find()][0])
l_headers.pop(0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  after removing the cwd from sys.path.


'_id'

In [200]:
for i in db.find():
    print(i)
    break

{'_id': ObjectId('6083462a93578a39deca6416'), '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, '10': 11, '11': 12, '12': 13, '13': 14, '14': 15, '15': 16, '16': 17, '17': 18, '18': 19, '19': 20, '20': 21, '21': 22, '22': 23, '23': 24, '24': 25, '25': 26, '26': 27, '27': 28, '28': 29, '29': 30, '30': 31, '31': 32, '32': 33, '33': 34, '34': 35, '35': 36, '36': 37, '37': 38, '38': 39, '39': 40, '40': 41, '41': 42, '42': 43, '43': 44, '44': 45, '45': 46, '46': 47, '47': 48, '48': 49, '49': 50, '50': 51, '51': 52, '52': 53, '53': 54, '54': 55, '55': 56, '56': 57, '57': 58, '58': 59, '59': 60, '60': 61, '61': 62, '62': 63, '63': 64, '64': 65, '65': 66, '66': 67, '67': 68, '68': 69, '69': 70, '70': 71, '71': 72, '72': 73, '73': 74, '74': 75, '75': 76, '76': 77, '77': 78, '78': 79, '79': 80, '80': 81, '81': 82, '82': 83, '83': 84, '84': 85, '85': 86, '86': 87, '87': 88, '88': 89, '89': 90, '90': 91, '91': 92, '92': 93, '93': 94, '94': 95, '95': 96, '96': 97, '97

## TRAER DATOS DE MONGO

In [212]:
list_serie=[]
for i in db.find():
    list_serie.append(pd.Series(i))
    
for i in range(len(list_serie)):
    if i>12:
        continue
    if i==0:
        df = pd.DataFrame(list_serie[i], columns=[l_headers[i]])
    else:
        df_aux = pd.DataFrame(list_serie[i], columns=[l_headers[i]])
        df = pd.concat([df, df_aux],axis=1)
df = df.reset_index().iloc[1:,1:]

## SQL SERVER

In [27]:
drivers = pyodbc.drivers()
print(drivers)

['ODBC Driver 17 for SQL Server']


In [48]:
server = 'sql-server' 
database = 'AdventureWorks2019' 
username = 'sa' 
password = 'Mi_Super_Mega_Clave_Sql_Server' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

## CREATE TABLE

In [116]:
create_table = """

CREATE TABLE new_table_users(ID INTEGER, NOMBRE VARCHAR(100));

"""

cursor.execute(create_table)
    

In [155]:
query = """

SELECT * FROM new_table_users

"""

pd.read_sql(query, cnxn)
    

,ID,NOMBRE
0,0,Juan
1,1,Raul
2,2,Roberto
3,3,Jose
4,4,Lucas
5,5,Pepe


## INSERT

In [ ]:
INSERTAMOS UN DATO EN LA TABLA CREADA

In [132]:
insert_data = """

INSERT new_table_users (ID, NOMBRE) 
VALUES (1, 'JUAN')

"""

cursor.execute(insert_data)

In [ ]:
INSERTAMOS EN UN BUCLE 

In [154]:
dict_nombres = {0:'Juan',1:'Raul',2:'Roberto',3:'Jose',4:'Lucas',5:'Pepe'}

for i in range(6):
    insert_data = """

    INSERT new_table_users (ID, NOMBRE) 
    VALUES ("""+str(i)+""", '"""+dict_nombres[i]+"""')

    """

    cursor.execute(insert_data)

## DELETE 

In [153]:
query_delete = """

    DELETE  new_table_users
    
"""
#pd.read_sql(query_delete, cnxn)
try:
    cursor.execute(query_delete)
    print('Se han eliminado correctamente los datos...')
except:
    print('Fallo el script al eliminar los datos...')

Se han eliminado correctamente los datos...


## RESTORE

In [160]:
query_bkp = """

    RESTORE DATABASE [AdventureWorks2019] FROM DISK = '/scripts/AdventureWorks2019.bak' WITH MOVE 'AdventureWorks2019' TO '/var/opt/mssql/data/AdventureWorks2019.mdf', MOVE 'AdventureWorks2019_log' TO '/var/opt/mssql/data/AdventureWorks2019_log.ldf', FILE = 1,  NOUNLOAD,  STATS = 5;

"""

cursor.execute(query_bkp)

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot perform a backup or restore operation within a transaction. (3021) (SQLExecDirectW)')

## JOIN

In [9]:
sql = """
    select AA.name as Schemas, BB.name as Tables
    from sys.schemas AA
    JOIN sys.tables BB ON AA.schema_id = BB.schema_id
"""

df_sql = pd.read_sql(sql, cnxn)

In [49]:
query = """

SELECT *
  FROM  Sales.Customer

"""

df_sql = pd.read_sql(query, cnxn)

In [50]:
df_sql

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,1,NaN,934.0,1,AW00000001,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2014-09-12 11:15:07.263
1,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263
2,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263
3,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263
4,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263
...,...,...,...,...,...,...,...
19815,30114,1985.0,1986.0,7,AW00030114,97154F3D-28F1-4B15-AE03-9518B781ECE3,2014-09-12 11:15:07.263
19816,30115,1987.0,1988.0,6,AW00030115,E4CF8FD5-30A4-4B8E-8FD8-47032E255778,2014-09-12 11:15:07.263
19817,30116,1989.0,1990.0,4,AW00030116,EC409609-D25D-41B8-9D15-A1AA6E89FC77,2014-09-12 11:15:07.263
19818,30117,1991.0,1992.0,4,AW00030117,6F08E2FB-1CD3-4F6E-A2E6-385669598B19,2014-09-12 11:15:07.263


## WHERE -> LIKE 

In [200]:
query_works = """

SELECT *
  FROM  Person.Person
  WHERE FirstName like 'Lucas%' 

"""

pd.read_sql(query_works, cnxn)

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,4823,IN,False,None,Lucas,None,Adams,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",58963024-7FD6-4BCA-96B5-6060D80204DF,2014-03-22
1,4812,IN,False,None,Lucas,None,Allen,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",7C3D5F53-7035-4F05-B269-416636F30FF7,2013-07-20
2,4786,IN,False,None,Lucas,None,Anderson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",A93FD29E-69E1-4306-861B-9F6BAEFC20DE,2013-09-05
3,15180,IN,False,None,Lucas,J,Bailey,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",1941BB31-E701-4926-A3DB-32E7D0BAC121,2013-11-16
4,4824,IN,False,None,Lucas,None,Baker,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",CD05EC4C-F2D7-42E8-AA4E-07D5B401DA6F,2013-09-13
5,13910,IN,False,None,Lucas,None,Barnes,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",EFC4479B-BBE4-4B3E-87BE-41976A7BAF9E,2013-09-13
6,15050,IN,False,None,Lucas,G,Blue,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",52467189-50D4-4594-91C4-4728C8AF8BA4,2012-01-14
7,13908,IN,False,None,Lucas,P,Bradley,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",3B8D55AF-8FC8-4A4F-A0AE-D1EE4E273FAA,2013-11-24
8,14161,IN,False,None,Lucas,S,Brooks,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",7CFC839D-E14B-45EF-B574-47B407CD8D9C,2013-10-20
9,4778,IN,False,None,Lucas,None,Brown,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",DB50CA6F-55BB-4EAB-9AFB-50A86B0FBD7A,2014-02-11


## GROUP BY 

In [173]:
query_works = """

SELECT top 999 PersonType, BusinessEntityID, FirstName, LastName
  FROM  Person.Person

"""

pd.read_sql(query_works, cnxn)

,PersonType,BusinessEntityID,FirstName,LastName
0,EM,1,Ken,Sánchez
1,EM,2,Terri,Duffy
2,EM,3,Roberto,Tamburello
3,EM,4,Rob,Walters
4,EM,5,Gail,Erickson
5,EM,6,Jossef,Goldberg
6,EM,7,Dylan,Miller
7,EM,8,Diane,Margheim
8,EM,9,Gigi,Matthew
9,EM,10,Michael,Raheem


In [188]:
funciones_agregacion = {0:'COUNT',1:'SUM',2:'AVG'}

FUNCION = 2

query_works = """

SELECT PersonType As PersonType, """+funciones_agregacion[FUNCION]+"""(BusinessEntityID) as """+funciones_agregacion[FUNCION]+"""
  FROM  Person.Person
  GROUP BY PersonType

"""

pd.read_sql(query_works, cnxn)

,PersonType,AVG
0,IN,11532
1,EM,137
2,SP,282
3,SC,1099
4,VC,1750
5,GC,2235


## UPDATES 

In [197]:
sql = """

    SELECT * 
      FROM [AdventureWorks2019].[Sales].[SalesOrderDetail]
     WHERE SalesOrderDetailID>1000
       AND SalesOrderID =  43894
       AND ProductID = 776
       
       
    """

pd.read_sql(sql, cnxn)

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43894,1001,7DF2-4E54-B0,4,776,1,2022.0,0.0,8088.0,60FC628F-89ED-441C-BE27-1B34038FC8B8,2011-07-01


In [196]:
sql = """

    UPDATE [AdventureWorks2019].[Sales].[SalesOrderDetail]
       SET  UnitPrice = 2022
     WHERE SalesOrderDetailID>1000
       AND SalesOrderID =  43894
       AND ProductID = 776
       
       
    """

cursor.execute(sql)

## BETWHEEN 

In [199]:
sql = """

    SELECT * FROM [AdventureWorks2019].[Sales].[SalesOrderDetail]
     WHERE SalesOrderDetailID>1000
       AND ProductID BETWEEN 800 AND 802
       
    """

pd.read_sql(sql, cnxn)

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,46607,10744,C1B5-40D6-8D,14,801,2,580.2538,0.02,7961.082136,74106DD3-AAB9-43BA-9927-199942A295D5,2012-05-30
1,46608,10794,ACF1-4852-B7,1,802,1,88.9320,0.00,88.932000,A425D435-C8CE-4B88-8F1E-52D2EA2084CE,2012-05-30
2,46616,11018,1264-43E6-B7,12,801,2,580.2538,0.02,6823.784688,3F491241-7E06-489E-9185-AA23AA4F786C,2012-05-30
3,46620,11057,FEE3-4994-8E,1,800,1,600.2625,0.00,600.262500,DDB2B92C-13A2-4735-A2B3-A818751B4130,2012-05-30
4,46620,11071,FEE3-4994-8E,1,801,1,600.2625,0.00,600.262500,70442ED8-A65C-45B1-ACB7-449DF960F5E9,2012-05-30
...,...,...,...,...,...,...,...,...,...,...,...
1296,74015,118765,None,1,800,1,1120.4900,0.00,1120.490000,D133C130-8869-441F-84F7-333C458E5C8E,2014-05-28
1297,74021,118780,None,1,800,1,1120.4900,0.00,1120.490000,89300771-59B9-457A-A9F4-7AE8B75BF3E4,2014-05-28
1298,74022,118782,None,1,800,1,1120.4900,0.00,1120.490000,12C2AC0D-E591-4682-80F0-53FB01EC0F07,2014-05-28
1299,74082,118929,None,1,800,1,1120.4900,0.00,1120.490000,4E97DF09-C120-4F0C-9D6B-597263371390,2014-05-29
